<a href="https://colab.research.google.com/github/WikiPol/GeoAI_and_DL_Seminar_Uni_HD_4/blob/main/data/TileConvertingAndLabeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tif-Tile Preprocessing and Labeling Tools
## This colab splits generated tif-tiles into smaller tiles, transforms them into jpgs and labels them for the intended YOLOv8 use.

The files need to be stored in Google Drive and a connection has to be established.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

First, the tiles have to be split into a size, that is better readable and has less labels per file. Adjust the input, output file and tile_size accordingly.

In [ ]:
# Tif data splitting
!pip install rasterio
import os
import glob
import numpy as np
import rasterio
from rasterio.windows import Window

# Input and output file specification
input_folder = "/content/drive/MyDrive/file_with_target_tifs"
output_folder = "/content/drive/MyDrive/GeoAI/output_file"

# Tile size specification
tile_size = 256

# Creating output file if not available
os.makedirs(output_folder, exist_ok=True)

# checking input folder
tif_files = glob.glob(os.path.join(input_folder, "*.tif"))
if not tif_files:
    print("No Tif files found")
else:
    print(f"{len(tif_files)} Tif-Files found")

# splitting process
for tif_path in tif_files:
    with rasterio.open(tif_path) as dataset:
        filename = os.path.splitext(os.path.basename(tif_path))[0]
        width, height = dataset.width, dataset.height

        # computing number of files
        num_tiles_x = (width + tile_size - 1) // tile_size
        num_tiles_y = (height + tile_size - 1) // tile_size

        print(f"{filename}: {num_tiles_x}x{num_tiles_y} Tiles")

        # Tile extraction
        for i in range(num_tiles_x):
            for j in range(num_tiles_y):
                x_off = i * tile_size
                y_off = j * tile_size
                window_width = min(tile_size, width - x_off)
                window_height = min(tile_size, height - y_off)
                window = Window(x_off, y_off, window_width, window_height)
                tile_data = dataset.read(window=window)

                # naming an save path setting
                tile_filename = f"{filename}_tile_{i}_{j}.tif"
                tile_path = os.path.join(output_folder, tile_filename)

                # saving tile
                with rasterio.open(
                    tile_path, "w", driver="GTiff", height=window_height, width=window_width,
                    count=dataset.count, dtype=dataset.dtypes[0], crs=dataset.crs,
                    transform=dataset.window_transform(window)
                ) as tile_dst:
                    tile_dst.write(tile_data)

                print(f"Tile saved: {tile_filename}")

print("End of process. Check your Google Drive")


Now the data is ready to be converted into jpgs. Adjust the input and output path accordingly (temp and output will be created if no corresponding directory is found).

In [ ]:
# tile transformation into pngs
import os
import cv2
import glob
import numpy as np
from osgeo import gdal, osr

# Input, Output and a temporal storage
input_folder = "/content/drive/MyDrive/GeoAI/TIFF_Tiles"
temp_folder = "/content/drive/MyDrive/GeoAI/temp_tiff" # for uncompressed tifs
output_folder = "/content/drive/MyDrive/GeoAI/yolo_images"

os.makedirs(temp_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# Transforming koordinates into px considering the EPSG
def lonlat_to_pixel_coords(lon, lat, geotransform, spatial_ref):
    target = osr.SpatialReference()
    target.ImportFromWkt(spatial_ref)

    # Transforming the reference system
    source = osr.SpatialReference()
    source.ImportFromEPSG(4326)  # WGS84
    transform = osr.CoordinateTransformation(source, target)
    x_geo, y_geo, _ = transform.TransformPoint(lon, lat)

    # Transforming into pixel coordinates
    origin_x, pixel_width, _, origin_y, _, pixel_height = geotransform
    pixel_x = int((x_geo - origin_x) / pixel_width)
    pixel_y = int((y_geo - origin_y) / pixel_height)
    return pixel_x, pixel_y

#  Accesing tif files
tif_files = glob.glob(os.path.join(input_folder, "*.tif"))
total_files = len(tif_files)

if total_files == 0:
    print("No tif files found")
else:
    print(f" {total_files} tif files found")

for idx, filepath in enumerate(tif_files):
    try:
        filename = os.path.splitext(os.path.basename(filepath))[0]
        dataset = gdal.Open(filepath, gdal.GA_ReadOnly)
        if dataset is None:
            print(f"Error opening: {filepath}")
            continue

        # Saving uncompressed tif
        temp_tif_path = os.path.join(temp_folder, filename + "_uncompressed.tif")
        driver = gdal.GetDriverByName("GTiff")
        driver.CreateCopy(temp_tif_path, dataset, 0)
        dataset = None

        dataset = gdal.Open(temp_tif_path)
        geotransform = dataset.GetGeoTransform()
        projection = dataset.GetProjection()

        img = dataset.ReadAsArray()
        if img is None or img.size == 0:
            print(f"Error loading {temp_tif_path}")
            continue

        # RGB [B4, B3, B2]
        if img.shape[0] >= 3:
            red, green, blue = img[0], img[1], img[2]
            rgb = np.stack([red, green, blue], axis=-1)
            bgr = rgb[..., ::-1]

            # Normalizing
            bgr = bgr.astype(np.float32)
            bgr = (bgr - bgr.min()) / (bgr.max() - bgr.min()) * 255
            bgr = bgr.astype(np.uint8)

            # Saving as jpg
            output_path = os.path.join(output_folder, filename + ".jpg")
            if cv2.imwrite(output_path, bgr):
                print(f"✅ [{idx + 1}/{total_files}] {filename}.jpg saved")
            else:
                print(f" Error saving {filename}.jpg")

        else:
            print(f"Error {filename}: Not enough bands: {img.shape}")

    except Exception as e:
        print(f"Error on {filepath}: {e}")

print("Conversion to jpgs finished")

Bevore the labeling process, adjust the paths input output and cvs path again. The cvs can be aquiered on Google Open Data and it includes the building blueprints that will be used to set the bounding boxes.

In [ ]:
# Labeling
import os
import pandas as pd
import rasterio
from shapely import wkt
from shapely.geometry import box
import numpy as np
from tqdm import tqdm

# Input and output paths
tile_folder = "/content/drive/MyDrive/GeoAI/TIFF_Tiles"
label_folder = "/content/drive/MyDrive/GeoAI/TIFF_Tiles_labels"
os.makedirs(label_folder, exist_ok=True)

# CSV path needs to be specified
csv_path = "/content/drive/MyDrive/GeoAI/open_buildings_wkt_polygon1.csv.gz"
df = pd.read_csv(csv_path)

# Cleaning the csv file
df["geometry"] = df["geometry"].astype(str).str.strip('"')
df["geometry"] = df["geometry"].apply(wkt.loads)

tiles = [os.path.join(tile_folder, f) for f in os.listdir(tile_folder) if f.endswith(".tif")]

print(f"🔍 {len(tiles)} TIFF-Kacheln gefunden.")

for tif_path in tqdm(tiles):
    try:
        with rasterio.open(tif_path) as src:
            bounds = box(*src.bounds)
            transform = src.transform

            # Chosing relevant geometries
            relevant_geoms = df[df["geometry"].apply(lambda g: g.intersects(bounds))]

            labels = []
            for _, row in relevant_geoms.iterrows():
                geom = row["geometry"].intersection(bounds)
                if geom.is_empty or not geom.is_valid:
                    continue

                minx, miny, maxx, maxy = geom.bounds

                x_center = ((minx + maxx) / 2 - bounds.bounds[0]) / (bounds.bounds[2] - bounds.bounds[0])
                y_center = 1 - ((miny + maxy) / 2 - bounds.bounds[1]) / (bounds.bounds[3] - bounds.bounds[1])
                box_width = (maxx - minx) / (bounds.bounds[2] - bounds.bounds[0])
                box_height = (maxy - miny) / (bounds.bounds[3] - bounds.bounds[1])

                class_id = 0  # relevant for label map in roboflow

                labels.append(f"{class_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}")

        # Saving in compatible yolov8 format
        label_path = os.path.join(label_folder, os.path.basename(tif_path).replace(".tif", ".txt"))
        with open(label_path, "w") as f:
            f.write("\n".join(labels))

    except Exception as e:
        print(f"Error on {tif_path}: {e}")

print("Label generation finished")


For the last step the data can be uploaded to Roboflow. The api key has to be generatet for the workspace specifically. Also the Ids and paths have to be adjustet.

In [ ]:
# uploads labeled files to roboflow
!pip install roboflow
from roboflow import Roboflow
import os
import glob
from osgeo import gdal

# Roboflow initializasion
rf = Roboflow(api_key="") #<- insert generatet key from your Roboflow account into brackets
workspaceId = 'geoai4' # workspace ID
projectId = 'geoai4_2' # project ID
project = rf.workspace(workspaceId).project(projectId)

input_folder = "/content/drive/MyDrive/GeoAI/yolo_images"
label_folder = "/content/drive/MyDrive/GeoAI/TIFF_Tiles_labels"
labelmap_path = "/content/drive/MyDrive/GeoAI/labelmap.txt"

files = glob.glob(os.path.join(input_folder, "*.jpg"))
total_files = len(files)

if total_files == 0:
    print("No files found")
else:
    print(f" {total_files} pictures found")

# Upload process
for idx, filepath in enumerate(files):
    #print(idx)
    filename = os.path.splitext(os.path.basename(filepath))[0]

    try:
        dataset = gdal.Open(filepath, gdal.GA_ReadOnly)
        if dataset is None:
            print(f"Error opening: {filepath}")
            continue

        print(project.single_upload(
            image_path=filepath,
            annotation_path=os.path.join(label_folder, filename + ".txt"),
            annotation_labelmap=labelmap_path
        ))
        print(f"✅ [{idx+1}/{total_files}] {filename} uploaded.")

    except Exception as e:
        error_message = str(e)
        if "already exists" in error_message:
            print(f"⏭️ [{idx+1}/{total_files}] {filename} already exists, skip.")
            continue
        else:
            print(f"Error on {filepath}: {e}")